In [2]:
import pandas as pd
import numpy as np
import imageio

W = 720
H = 480
work_dir = '/home/shen/BU_learn/EC720/Final_project'
label_excel_dir = work_dir + '/raw_data/ut-interaction_labels_110912.xls'
video_dir = []
for i in range(10):
    video_dir.append(work_dir + '/set_1/seq%d.avi'%(i+1))

In [4]:
fn = np.zeros(10)
for i in range(10):
    vid = imageio.get_reader(video_dir[i],  'ffmpeg')
    fn[i] = vid.get_length()

In [3]:
df = pd.read_excel(label_excel_dir,header=1)
df = df.dropna(subset=['Main actor (left=0, right=1)'])
sf = df[df['sequence #'] == 'seq%d'%1]
sf

,Set,sequence #,Activity Class ID,Starting Frame #,Ending Frame #,X-Cord,Y-Cord,X-Cord.1,Y-Cord.1,"Main actor (left=0, right=1)"
0,1.0,seq1,4.0,762.0,841.0,220.0,113.0,620.0,410.0,1.0
1,1.0,seq1,2.0,919.0,1008.0,185.0,90.0,673.0,423.0,1.0
2,1.0,seq1,1.0,1064.0,1207.0,225.0,105.0,655.0,420.0,-1.0
3,1.0,seq1,3.0,1311.0,1408.0,210.0,122.0,433.0,422.0,0.0
4,1.0,seq1,0.0,1408.0,1562.0,225.0,110.0,655.0,423.0,-1.0
5,1.0,seq1,5.0,1593.0,1675.0,200.0,110.0,640.0,410.0,1.0


In [65]:
for i in range(10):
    sf = df[df['sequence #'] == 'seq%d'%(i+1)]
    ground_truth = np.ones(int(fn[i]))
    ground_truth = ground_truth * 6
    for n, row in sf.iterrows():
        start = int(row['Starting Frame #'])
        end = int(row['Ending Frame #'])
        cid = row['Activity Class ID']
        ground_truth[start:end] = cid
    print('label_array lenth of seq_%d is: %d'%(i+1,ground_truth.shape[0]))
    np.save('array/Y_seq_%d.npy'%(i+1),ground_truth)

label_array lenth of seq_1 is: 2031
label_array lenth of seq_2 is: 1535
label_array lenth of seq_3 is: 2096
label_array lenth of seq_4 is: 1242
label_array lenth of seq_5 is: 1816
label_array lenth of seq_6 is: 1458
label_array lenth of seq_7 is: 2168
label_array lenth of seq_8 is: 2378
label_array lenth of seq_9 is: 4013
label_array lenth of seq_10 is: 2168


In [9]:
from skimage.io import imread,imshow,imsave
idx = 0
for i in range(10):
    sf = df[df['sequence #'] == 'seq%d'%(i+1)]
    img = imread('/home/shen/BU_learn/EC720/Final_project/set_1/seq%dbg.jpg'%(i+1))
    print(img.shape)
    sample_num = sf.shape[0]
    for j in range(sample_num):
        label = sf['Activity Class ID'][idx]
        x1 = int(sf['X-Cord'][idx])
        x2 = int(sf['X-Cord.1'][idx])
        y1 = int(sf['Y-Cord'][idx])
        y2 = int(sf['Y-Cord.1'][idx])
        print(y1,y2,)
        crop = img[y1:y2,x1:x2,:]
        filename = '/home/shen/BU_learn/EC720/Final_project/segmented_set1/%d_%d_%d.jpg'%(idx,i+1,label)
        imsave(filename,crop)
        idx+=1

(480, 720, 3)
113 410
90 423
105 420
122 422
110 423
110 410
(480, 720, 3)
100 400
108 400
108 400
102 400
102 390
115 390
(480, 720, 3)
135 416
140 433
145 423
143 414
149 440
142 442
(480, 720, 3)
150 433
150 428
145 432
143 432
153 428
147 428
(480, 720, 3)
144 404
137 410
140 411
136 405
136 413
138 414
(480, 720, 3)
155 386
160 400
160 400
168 387
170 390
(480, 720, 3)
160 400
160 400
150 400
155 400
160 390
160 390
(480, 720, 3)
155 400
160 395
166 400
165 400
165 400
165 385
165 390
(480, 720, 3)
126 370
133 366
134 377
131 380
135 360
125 365
(480, 720, 3)
230 447
222 445
233 450
220 450
215 455
220 455


/home/shen/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /home/shen/BU_learn/EC720/Final_project/segmented_set1/46_8_3.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)
/home/shen/anaconda3/lib/python3.6/site-packages/skimage/io/_io.py:132: UserWarning: /home/shen/BU_learn/EC720/Final_project/segmented_set1/50_9_2.jpg is a low contrast image
  warn('%s is a low contrast image' % fname)


In [48]:
img[1:15,2:70,2]

array([36], dtype=uint8)

In [113]:
arr_dir = work_dir + '/src/array'
ds_dir = work_dir + '/src/array_with_undersampleing'

for i in range(10):
    label_dir = arr_dir + '/Y_seq_%d.npy'%(i+1)
    data_dir = arr_dir + '/X_seq_%d.npy'%(i+1)
    
    label = np.load(label_dir)
    data = np.load(data_dir)
    print(data.shape)

    pos_num = np.sum(label==6)
    remain = pos_num//7

    label = np.load(label_dir)
    neg_label_idx = np.argwhere(label==6)
    obj = np.squeeze(neg_label_idx)
    np.random.shuffle(obj)
    split_idx = np.split(obj,[remain])

    new_label = np.delete(label, split_idx[1], axis=None)
    new_data = np.delete(data, split_idx[1], axis=0)

    print(new_label.shape)
    print(new_data.shape)    
    
    new_label_dir = ds_dir + '/Y_seq_%d.npy'%(i+1)
    new_data_dir = ds_dir + '/X_seq_%d.npy'%(i+1)
    np.save(new_label_dir,new_label)
    np.save(new_data_dir,new_data)

(2031, 480, 720)
(842,)
(842, 480, 720)
(1535, 480, 720)
(653,)
(653, 480, 720)
(2096, 480, 720)
(805,)
(805, 480, 720)
(1242, 480, 720)
(645,)
(645, 480, 720)
(1816, 480, 720)
(686,)
(686, 480, 720)
(1458, 480, 720)
(642,)
(642, 480, 720)
(2168, 480, 720)
(770,)
(770, 480, 720)
(2378, 480, 720)
(853,)
(853, 480, 720)
(4013, 480, 720)
(1053,)
(1053, 480, 720)
(2168, 480, 720)
(681,)
(681, 480, 720)


In [114]:
print(new_label)

[ 6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.
  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.
  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.
  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.  6.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.  0.
  6.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.
  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.
  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.  4.
  4.  4.  4.  4.  4.  6.  6.  6.  6.  5.  5.  5.  5.  5.  5.  5.  5.  5.
  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5.  5